## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details.

In [1]:
import sqlite3
import nltk
import random
import numpy as np
from collections import Counter, defaultdict
import re
from nltk.corpus import stopwords
from string import punctuation
from nltk.tokenize import word_tokenize
import csv
import pandas as pd

# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns

In [2]:
convention_db = sqlite3.connect('2020_Conventions.db')
convention_cur = convention_db.cursor()

In [3]:
#see what in the table
sql_query = """SELECT name FROM sqlite_master  
  WHERE type='table';"""
convention_cur.execute(sql_query)
print("List of tables\n")
print(convention_cur.fetchall())

List of tables

[('conventions',)]


### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

In [4]:
convention_data = []

# fill this list up with items that are themselves lists. The 
# first element in the sublist should be the cleaned and tokenized
# text in a single string. The second element should be the party. 

convention_cur.execute('SELECT * FROM conventions')
rows = convention_cur.fetchall()
for row in rows :
     convention_data.append(row)



In [5]:
#could be a single function with 2 parameters but got lazy and copied and pasted
description=[]
political_s=[]


for i in range(len(convention_data)):        
        party_name=convention_data[i][0]
        sub_list=political_s.append(party_name)

for i in range(len(convention_data)):        
        text=convention_data[i][5]
        sub_list=description.append(text)        

In [6]:
# functions to clean should of imported above but had some errors to fix & test
sw = stopwords.words("english")
def remove_punctuation(text) : 
    return str(re.sub(r'[^\w\s]', '', text))
     #(str(text.replace('[^\w\s]','')))
def tokenize(text) : 
    """ Splitting on whitespace rather than the book's tokenize function. That 
        function will drop tokens like '#hashtag' or '2A', which we need for Twitter. """    
    word = re.split('\s+', text)
    return  word 

def low(text):
    return(str(text.lower()))

def remove_stop(tokens):
    text_tokens = word_tokenize(tokens)
    tokens_without_sw = [word for word in text_tokens if not word in stopwords.words()]
    return tokens_without_sw

def map_funcs(obj, func_list):
    return [func(obj) for func in func_list]

In [7]:
len(description)

2541

In [8]:
#test2=low(description[1])
# Run time O(N)
clean =[]
for i in range(2541):
   # for words in range(len (description[i])):
    temp=low(str(description[i]) )   
    temp=remove_stop(str(temp))
    temp=remove_punctuation(str(temp))
    #tokenization can been done in the loop down to where we extract feature words 
    clean.append(temp)

In [ ]:
#proof of concept that running it works
print(clean)

In [9]:

store1=zip((clean),(political_s))
store1=list(store1)



In [ ]:
print(store1)

Let's look at some random entries and see if they look right. 

In [10]:
random.choices(store1,k=10)

[('choosing preventative mastectomy hardest decision make  supporting president trump  protect daughter children  future easiest ',
  'Republican'),
 ('pledge allegiance flag united states america republic stands  nation god  indivisible  liberty justice speaker 5   0658   foreign language 000658   myron lizer  vice president navajo nation  give address majestic rock spires shiprock  tsé bitʼaʼí  rock wings  dene   ancestral leaders sought govern lead nation nation  sought lead promises life children  children  successful rest america  nations  host land   creator knew  time  opportunity appeal heaven   invited american dream   years fought congressional battles past congressmen senators  part broken system ignored ',
  'Republican'),
 ('danny burch back home  yemen horrible situation  yesterday  united states government secured release caitlan coleman  joshua boyle  children  today  bringing home american citizen  news turkey released prisoner united states ',
  'Republican'),
 ('70 y

In [ ]:
with open('politics_file2', 'w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(store1)

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [11]:

word_cutoff = 5

tokens = [w for t, p in store1 for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2103 as features in the model.


In [ ]:
#ignore tester block with sets
test=" quick quick brown fox"
li = list(test.split(" "))

 
# List2
List2 = ['quick','fox','jumps']

samevalue=set(li) & set(List2) # pulls everything the same
type(samevalue)


In [13]:

def conv_features(text,fw) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    
    
    
    # Your code here
    li = list(text.split(" "))
    same_values=set(li) & set(fw)
    ret_dict = dict()
    ret_dict = dict.fromkeys(same_values, True)
    return(ret_dict)
   

In [15]:
# no idea why people is missing must of been in cleaning
conv_features("people are american in america",feature_words)

{'america': True, 'american': True}

In [14]:
assert(len(feature_words)>0)
assert(conv_features("donald is the president",feature_words)==
       {'donald':True,'president':True})
assert(conv_features("people are american in america",feature_words)==
                     {'america':True,'american':True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [16]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in store1]

In [17]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [18]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.528


In [19]:
classifier.show_most_informative_features(25)

Most Informative Features
                   china = True           Republ : Democr =     27.1 : 1.0
                   votes = True           Democr : Republ =     23.8 : 1.0
             enforcement = True           Republ : Democr =     21.5 : 1.0
                 destroy = True           Republ : Democr =     19.2 : 1.0
                freedoms = True           Republ : Democr =     18.2 : 1.0
                 climate = True           Democr : Republ =     17.8 : 1.0
                supports = True           Republ : Democr =     17.1 : 1.0
                   crime = True           Republ : Democr =     16.1 : 1.0
                   media = True           Republ : Democr =     15.8 : 1.0
                 beliefs = True           Republ : Democr =     13.0 : 1.0
               countries = True           Republ : Democr =     13.0 : 1.0
                 defense = True           Republ : Democr =     13.0 : 1.0
                  defund = True           Republ : Democr =     13.0 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

The republican party was higher than the deomcratic party on words. I guess that the Republicans use more unique words which gives them higher presence.



## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [20]:
cong_db = sqlite3.connect("congressional_data.db")
cong_cur = cong_db.cursor()

In [21]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [22]:
# loop inside the array and extract the values we want.
tweet_data2 = [[i[2].decode('utf-8'),i[1]] for i in results]

In [23]:
tweet_data1=[]
politic_name=[]
for i in range (664656):
    temp=low(str(tweet_data2[i][0]) )   
    temp=remove_punctuation(str(temp))
    sub_list=tweet_data1.append(temp)
# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.
for i in range (664656):
    tempor=tweet_data2[i][1]
    sub_list=politic_name.append(tempor)

In [24]:
store2=zip((tweet_data1),(politic_name))
store2=list(store2)

In [25]:
tweet_data=store2

There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [26]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

In [27]:

for tweet, party in tweet_data_sample :
    estimated_party = classifier.classify(conv_features(tweet,feature_words))
    # Fill in the right-hand side above with code that estimates the actual party
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")
    

Here's our (cleaned) tweet: earlier today i spoke on the house floor abt protecting health care for women and praised ppmarmonte for their work on the central coast httpstcowqgtrzt7vv
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: go tribe rallytogether httpstco0nxutfl9l5
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: apparently trump thinks its just too easy for students overwhelmed by the crushing burden of debt to pay off student loans trumpbudget httpstcockyqo5t0qh
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: were grateful for our first responders our rescue personnel our firefighters our police and volunteers who have been working tirelessly to keep people safe provide muchneeded help while putting their own lives on the line

httpstcoezpv0vmiz3
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: lets make it e

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [28]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
   
    # get the estimated party
    estimated_party = classifier.classify(conv_features(tweet,feature_words))
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [29]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 3353, 'Democratic': 925}),
             'Democratic': defaultdict(int,
                         {'Republican': 4323, 'Democratic': 1401})})

### Reflections

Seems like we have a bias towards republican identification witht he classifier. Alot of Democrats got categorized as Republican due to the rhetoric. Mabey Democrats reply back to Republican with the same language to make a remark. Because we can also see that Republicans havent got labeled as much as Democrats. Also I guess we have to keep size of party in account.